In [ ]:
import os
import datetime
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.optimize
import scipy.interpolate

import pydicom

In [ ]:
data_root = r'S:\Physics\Programming\data\MVISO'

In [ ]:
data_record = glob(os.path.join(data_root, 'iView*.xlsx'))[0]
dicom_files = np.array(glob(os.path.join(data_root, '*.dcm')))

In [ ]:
record = pd.read_excel(data_record, skiprows=4)
timestamps_initial = record['Datetime']
timestamps = timestamps_initial[timestamps_initial.notnull()].values
gantry = record['Gantry'][timestamps_initial.notnull()].values
colimator = record['Col'][timestamps_initial.notnull()].values
turntable = record['TT'][timestamps_initial.notnull()].values
beam = record['Energy'][timestamps_initial.notnull()].values

In [ ]:
datasets = np.array([
    pydicom.read_file(dicom_file, force=True)
    for dicom_file in dicom_files
])

In [ ]:
acquisition_datetimes = np.array([
    datetime.datetime.strptime(dataset.AcquisitionDate + dataset.AcquisitionTime, '%Y%m%d%H%M%S.%f')
    for dataset in datasets
], dtype=np.datetime64)

In [ ]:
diff_map = np.abs(acquisition_datetimes[None,:] - timestamps[:, None]) < np.timedelta64(2, 's')
timestamp_index, acquisition_index = np.where(diff_map)

In [ ]:
assert len(set(acquisition_index)) == len(acquisition_index)
assert len(acquisition_index) == len(acquisition_datetimes)

In [ ]:
datasets = datasets[acquisition_index]
dicom_files = dicom_files[acquisition_index]
timestamps = timestamps[timestamp_index]
gantry = gantry[timestamp_index]
colimator = colimator[timestamp_index]
turntable = turntable[timestamp_index]
beam = beam[timestamp_index]

acquisition_datetimes = np.array([
    datetime.datetime.strptime(dataset.AcquisitionDate + dataset.AcquisitionTime, '%Y%m%d%H%M%S.%f')
    for dataset in datasets
], dtype=np.datetime64)

diff_map = np.abs(acquisition_datetimes[None,:] - timestamps[:, None]) < np.timedelta64(2, 's')
timestamp_index, acquisition_index = np.where(diff_map)

assert np.all(timestamp_index == acquisition_index)

In [ ]:
pixel_arrays = np.array([
    dataset.pixel_array
    for dataset in datasets
], copy=True)

pixel_arrays = 1 - pixel_arrays/2**16

In [ ]:
axis_distance = np.arange(-512, 512)/4

initial_mask_distance = 20  # mm

first = np.where(axis_distance >= -initial_mask_distance)[0][0]
last = np.where(axis_distance > initial_mask_distance)[0][0]

mask = slice(first, last)

axis_distance = axis_distance[mask]

masked_arrays = np.array([
    pixel_array[mask, mask]
    for pixel_array in pixel_arrays
])

In [ ]:
axis_distance

In [ ]:
dx = 0.05
interpolated_distances = np.arange(-initial_mask_distance, initial_mask_distance+dx, dx)

xx, yy = np.meshgrid(interpolated_distances, interpolated_distances)
xx_flat = np.ravel(xx)
yy_flat = np.ravel(yy)
# interpolated_distances

In [ ]:
interpolated_distances

interpolation = scipy.interpolate.RectBivariateSpline(axis_distance, axis_distance, masked_arrays[0], kx=1, ky=1)

interpolated_image_flat = interpolation.ev(yy_flat, xx_flat)
interpolated_image = np.reshape(interpolated_image_flat, np.shape(xx))

In [ ]:
scipy.interpolate.RectBivariateSpline(axis_distance, axis_distance, masked_arrays[0], kx=1, ky=1).ev

In [ ]:
# xx, yy = np.meshgrid(axis_distance, axis_distance)

In [ ]:
def show_image(pixel_array):
    plt.pcolormesh(interpolated_distances, interpolated_distances, pixel_array, clim=[0, 1])
    plt.colorbar()
    plt.axis('equal')
    
show_image(interpolated_image)

In [ ]:
square_field_side_length = 20  # mm
penumbra_width = 3  # mm
ball_bearing_diameter = 8 # mm

In [ ]:
# show_image(masked_arrays[0])

In [ ]:
def show_image_with_square(image, centre, edge_length):
    x = centre[0]
    y = centre[1]
    
    plt.plot(
        [x - edge_length/2, x - edge_length/2, x + edge_length/2,  x + edge_length/2, x - edge_length/2],
        [y - edge_length/2, y + edge_length/2, y + edge_length/2,  y - edge_length/2, y - edge_length/2],
        'k', lw=2
    )
    
    show_image(image)
    plt.show()
    

show_image_with_square(interpolated_image, [0,3], square_field_side_length)

In [ ]:
def mean_inside_square_take_outside(x, y, side_length, image):
    is_inside_square = (
        (xx > x - side_length/2) & 
        (xx < x + side_length/2) &
        (yy > y - side_length/2) &
        (yy < y + side_length/2)
    )
    
    return np.mean(image[is_inside_square]) - np.mean(image[np.invert(is_inside_square)])

In [ ]:
def create_field_minimisation(square_field_side_length, image_to_search):
    def field_to_minimise(coords):
        x = coords[0]
        y = coords[1]
        side_length = square_field_side_length
        image = image_to_search
        
        return -mean_inside_square_take_outside(x, y, side_length, image)
    
    return field_to_minimise


def create_print_func(square_field_side_length, image_to_search):
    def print_fun(centre, f, accepted):
        show_image_with_square(image_to_search, centre, square_field_side_length)
        
    return print_fun


to_minimise = create_field_minimisation(square_field_side_length, interpolated_image)
print_fun = create_print_func(square_field_side_length, interpolated_image)

In [ ]:
results = scipy.optimize.basinhopping(to_minimise, [0,0], T=0.01, niter=100, stepsize=1)
initial_centre = results.x

print(initial_centre)

plt.figure(figsize=(15,15))
show_image_with_square(interpolated_image, initial_centre, 18)

In [ ]:
np.shape(interpolated_image)

In [ ]:
plt.figure(figsize=(10,10))

plt.plot([centre[0]-10, centre[0]-10], [0,0.7])
plt.plot([centre[0]+10, centre[0]+10], [0,0.7])

for i in range(300, 500):

    plt.plot(interpolated_distances, interpolated_image[i,:])
    plt.plot(centre[0] - interpolated_distances, interpolated_image[i,:])

In [ ]:
plt.plot(interpolated_distances, interpolated_image[400,:])
plt.plot(centre[0] - interpolated_distances, interpolated_image[400,:])

In [ ]:
average_cross_profile

In [ ]:
average_cross_profile = np.mean(interpolated_image[300:500,:], axis=0)
maximum = np.max(average_cross_profile)

mid = maximum / 2

In [ ]:
scipy.interpolate.interp1d(interpolated_distances, average_cross_profile)([centre[0]-10, centre[0]+10])

In [ ]:
mid

In [ ]:
plt.plot(interpolated_distances, np.mean(interpolated_image[300:500,:], axis=0))

In [ ]:
results

In [ ]:
centre

In [ ]:


    
for pixel_array in masked_arrays:
    show_image(pixel_array)